In [1]:
# load packages
import os
import glob
import tensorflow as tf
from input_server import imgs_input_fn, build_model
import numpy as np
import pandas as pd
from scipy.stats import norm
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error
from helpers_trans import find_closest_element

In [2]:
# path of model checkpoints
checkpoint_path = '../../bdd100k_test_data/models/10092020_adadelta/'
extracted_coefficients_directory = '../../bdd100k_test_data/extracted_coefficients/10092020'
model_dir = '../../bdd100k_test_data/models/10092020_adadelta'
val_path = '../../data/tfrecords/03082020/val_5000.tfrecords'
shard_path = '../../data/tfrecords/shards/07092020'
shard_files = glob.glob(os.path.join(shard_path, "*.tfrecords")) 

In [3]:
# actually also need all train yaws...
val_all_true_yaws_path = '../../data/tfrecords/03082020/val_yaw_original.csv'
train_all_true_yaws_path = '../../data/tfrecords/03082020/train_yaw_original.csv'
val_yaws = pd.read_csv(val_all_true_yaws_path)
train_yaws = pd.read_csv(train_all_true_yaws_path)

In [4]:
n_val = val_yaws.shape[0]
n_train = train_yaws.shape[0]

In [5]:
n_val

114619

## 0. Get weights $\beta$ of the last hidden layer 

In [6]:
# build model
Input = tf.keras.layers.Input(shape=(640, 360, 3,), name='image')
x = Conv2D(24, kernel_size=(5, 5), activation='swish', strides=(2, 2))(Input)
x = BatchNormalization()(x)
x = Conv2D(36, kernel_size=(5, 5), activation='swish', strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(36, kernel_size=(3, 3), activation='swish', strides=(2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(48, kernel_size=(3, 3), activation='swish')(x)
x = BatchNormalization()(x)
x = Conv2D(64, kernel_size=(3, 3), activation='swish')(x)
x = Dense(128)(x)
x = Dropout(0.8)(x)
x = Dense(96)(x)
x = Dropout(0.8)(x)
x = Dense(64)(x)
x = Dropout(0.8)(x)
x = Dense(32)(x)
x = Dropout(0.8)(x)
x = Dense(16)(x)
Output = Dense(1, name = 'output_layer')(x)

keras_model = tf.keras.models.Model(
      inputs = [Input], outputs = [Output])

# update model with weights
keras_model.load_weights(tf.train.latest_checkpoint(checkpoint_path))

keras_model.compile(
    loss = 'mae',
    optimizer = keras.optimizers.Adadelta(learning_rate = 0.0005, rho = 0.95, epsilon = 1e-07, name = 'Adadelta'),
    metrics=['mse', 'mae'])

# to inspect weights:
#i = 0 
#for layer in keras_model.layers: 
#    i += 1
#    if i == 16:
#        print(layer.get_config(), layer.get_weights())

# get list of all variables in checkpoint file
# tf.train.list_variables(tf.train.latest_checkpoint(checkpoint_path))

In [7]:
i = 0
for layer in keras_model.layers: 
    i += 1
    if i == 20:
        beta = layer.get_weights()
        #print(layer.get_config(), layer.get_weights())
# last element of beta is bias and is set to zero
beta = beta[0]
bias = beta[1]
np.savetxt(str(extracted_coefficients_directory +"/beta.csv"), beta, delimiter=",")
np.savetxt(str(extracted_coefficients_directory +"/bias.csv"), beta, delimiter=",")

In [8]:
i = 0
for layer in keras_model.layers: 
    i += 1
    print(layer.get_config())

{'batch_input_shape': (None, 640, 360, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'image'}
{'name': 'conv2d', 'trainable': True, 'dtype': 'float32', 'filters': 24, 'kernel_size': (5, 5), 'strides': (2, 2), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'swish', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'batch_normalization', 'trainable': True, 'dtype': 'float32', 'axis': ListWrapper([3]), 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer': {'class_name': 'Ones', 'config': {}}, 'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}}, 'moving_variance

## 1. Get outputs $B_{\zeta}(x_i) $ from last hidden layer

In [9]:
B_zeta_estimator = build_model(model_dir, checkpoint_path)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '../../bdd100k_test_data/models/10092020_adadelta', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


 Validation set predictions $B_{\zeta}$

In [10]:
cust_train_input_fn_val = lambda: imgs_input_fn(val_path)
pred_results_b_zeta_val = B_zeta_estimator.predict(input_fn = cust_train_input_fn_val)
#B_zeta_predictions = next(pred_results_b_zeta)
# like this we get all predictions... but not so pretty
val_B_zeta_predictions_all = []
for i in range(0,int(n_val/2886) + 1):
    val_B_zeta_predictions = next(pred_results_b_zeta_val)
    val_B_zeta_predictions_all.append(val_B_zeta_predictions['dense_9'])
val_B_zeta_predictions_all = np.array(val_B_zeta_predictions_all)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../../bdd100k_test_data/models/10092020_adadelta/model.ckpt-23433
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Training set predictions $B_{\zeta}$

In [11]:
train_B_zeta_predictions_all = []
cust_train_input_fn_train = lambda: imgs_input_fn(shard_files)
pred_results_b_zeta_train = B_zeta_estimator.predict(input_fn = cust_train_input_fn_train)
for i in range(0,int(n_train/2886) + 1):
    train_B_zeta_predictions = next(pred_results_b_zeta_train)
    train_B_zeta_predictions_all.append(train_B_zeta_predictions['dense_9'])
train_B_zeta_predictions_all = np.array(train_B_zeta_predictions_all)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../../bdd100k_test_data/models/10092020_adadelta/model.ckpt-23433
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [12]:
# reshape and cut off duplicates
ts = train_B_zeta_predictions_all.shape
train_B_zeta_predictions_all = train_B_zeta_predictions_all.reshape(ts[0]*ts[1]*ts[2], ts[3])[:n_train]
vs = val_B_zeta_predictions_all.shape
val_B_zeta_predictions_all = val_B_zeta_predictions_all.reshape(vs[0]*vs[1]*vs[2], vs[3])[:n_val]

In [13]:
val_B_zeta_predictions_all.shape, n_val

((114619, 16), 114619)

In [14]:
# get predictions
pred_dnn_val = val_B_zeta_predictions_all.dot(beta)
pred_dnn_train = train_B_zeta_predictions_all.dot(beta)

In [15]:
for name,dict_ in B_zeta_predictions.items():
    print(name)

NameError: name 'B_zeta_predictions' is not defined

In [16]:
# save outputs
val_B_zeta_pred = val_B_zeta_predictions_all
train_B_zeta_pred = train_B_zeta_predictions_all
np.savetxt(str(extracted_coefficients_directory +"/B_zeta_predictions_val.csv"), 
           val_B_zeta_pred,
           delimiter=",")
np.savetxt(str(extracted_coefficients_directory +"/B_zeta_predictions_train.csv"), 
           train_B_zeta_pred,
           delimiter=",")

In [ ]:
B_zeta_pred.shape

In [ ]:
# get predictions:
pred_dnn = B_zeta_pred.dot(beta)

## Get true z

In [17]:
val_transformed_yaws_path = '../../data/tfrecords/03082020/val_yaw_transformed.csv'
train_transformed_yaws_path = '../../data/tfrecords/03082020/train_yaw_transformed.csv'
true_z_val = np.genfromtxt(val_transformed_yaws_path, delimiter=',')
true_z_train = np.genfromtxt(train_transformed_yaws_path, delimiter=',')

In [ ]:
true_z_train.shape

## check if predictions are more or less correct (in case the nn has reinitialized the weights...)
yes, confirms with training and test errors
1. Mean squared error

In [18]:
# get densities
density_path = '../../data/density/03082020/fast_kde_density.csv'
density = pd.read_csv(density_path)
# transform predictions back to yaw
train_yaw_pred_nn_norm = norm.cdf(pred_dnn_train).reshape(pred_dnn_train.shape[0],)
train_yaw_pred_nn = np.array([density.loc[find_closest_element(y_i,density['cdf']),'axes'] 
                                 for y_i in train_yaw_pred_nn_norm])
np.savetxt(str(extracted_coefficients_directory +"/train_yaw_pred_nn.csv"), train_yaw_pred_nn, delimiter=",")

# transform predictions back to yaw
val_yaw_pred_nn_norm = norm.cdf(pred_dnn_val).reshape(pred_dnn_val.shape[0],)
val_yaw_pred_nn = np.array([density.loc[find_closest_element(y_i,density['cdf']),'axes'] 
                                 for y_i in val_yaw_pred_nn_norm])
np.savetxt(str(extracted_coefficients_directory +"/val_yaw_pred_nn.csv"), val_yaw_pred_nn, delimiter=",")

In [ ]:
val_yaw_pred_nn.shape

In [19]:
# validation error
mse_val = mean_squared_error(val_yaws[:val_yaw_pred_nn.shape[0]], val_yaw_pred_nn)
# training error
mse_train = mean_squared_error(train_yaws[:train_yaw_pred_nn.shape[0]], train_yaw_pred_nn)
print('Performance on the validation set: ' 
      + str(mse_val) 
      +' \nPerformance on the training set:' 
      + str(mse_train))

Performance on the validation set: 0.020078856650268206 
Performance on the training set:0.01981448799572632


2. Mean absolute error

In [20]:
# validation error
mae_val = mean_absolute_error(val_yaws, val_yaw_pred_nn)
# training error
mae_train = mean_absolute_error(train_yaws, train_yaw_pred_nn)
print('Performance on the validation set: ' 
      + str(mae_val) 
      +' \nPerformance on the training set:' 
      + str(mae_train))

Performance on the validation set: 0.10048306878779562 
Performance on the training set:0.0995873184381412
